# PowerShell User Group innsalzach 04.02.2020

## PowerShell an Microsoft Graph

### What is Microsoft Graph?
---

Microsoft Graph holds all the data that we generate in Microsoft 365, Windows 10 and Enterprise Mobility + Security.

What are these Services:

- Microsoft 365 services: Delve, Excel, Microsoft Bookings, Microsoft Teams, OneDrive, OneNote, Outlook/Exchange, Planner, SharePoint, Workplace Analytics.
    
- Enterprise Mobility and Security services: Advanced Threat Analytics, Advanced Threat Protection, Azure Active Directory, Identity Manager, and Intune.
    
- Windows 10 services: activities, devices, notifications, Universal Print (preview).
    
- Dynamics 365 Business Central.


![Microsoft Graph](https://docs.microsoft.com/en-us/graph/images/microsoft-graph.png)

It is a rest api that is accessible over [https://graph.microsoft.com](https://graph.microsoft.com)

Whant to read more? Use this link: [Microsoft Graph](https://docs.microsoft.com/en-us/graph/overview)

### What is ADAL / MSAL
---

ADAL = Azure Active Directory Authentication Library

MSAL = Microsoft Authentication Library

Both libraries handle the Authentication against Azure Active Directory and provide you an authentication token.

MSAL is the recommended authentication library for use with the Microsoft identity platform. 

No new features will be in ADAL and It was deprecated at the 30th July 2020 and will get fixes until 30th June 2022.

Here are some advantages why you should use MSAL instead of ADAL:
- Authentication with broader set of identities (Azure AD, Microsoft Account, Azure AD B2C Social Accounts)
- Best single-sign-on experience
- Support of conditional access is easier

If you need guidance of migrating an existing ADAL app go to this link [MSAL Migration Guide](https://docs.microsoft.com/de-de/azure/active-directory/develop/msal-migration)

There is a PowerShell Module for using MSAL and ADAL.

- ADAL: ADAL.PS [ADAL.PS PowerShell Gallery](https://www.powershellgallery.com/packages/ADAL.PS/3.19.4.2)
- MSAL: MSAL.PS [MSAL.PS PowerShell Gallery](https://www.powershellgallery.com/packages/MSAL.PS/4.21.0.1)

Please use MSAL.PS for new projects. We will use It later. It is also PowerShell 7 compatible.

Questions?

### How do I authenticate against Microsoft Graph?
---


In [ ]:
# Install the module if not already available
if (Get-Module MSAL.PS -ListAvailable) {
    Write-Output "MSAL.PS is already available."   
}
else {
    Install-Module MSAL.PS -AcceptLicense -Force
}

# Optinonally install JWTDetails

if (Get-Module JWTDetails -ListAvailable) {
    Write-Output "JWTDetails is already available."   
}
else {
    Install-Module JWTDetails -Force
}

In [ ]:
# Get a Token (With Client Secret)
$cred = Import-Clixml -Path C:\Temp\clientsecret.xml

$splat = @{
    ClientId     = "d520b7d0-da08-4026-8962-6c59a74d2238" 
    TenantId     = "f8ddd457-53bf-4bcd-bd31-c1a30d7c78d7" 
    ClientSecret = (ConvertTo-SecureString -String $cred.GetNetworkCredential().Password -AsPlainText -Force)
}
$Result = Get-MsalToken @splat

In [ ]:
# Get a Token (With Device Login)(Be sure you have public client flow enabled on the Azure AD application)
$splat = @{
    ClientId     = "d520b7d0-da08-4026-8962-6c59a74d2238" 
    TenantId     = "f8ddd457-53bf-4bcd-bd31-c1a30d7c78d7" 
    DeviceCode  = $true
}
$Result = Get-MsalToken @splat

In [ ]:
# Inspect the result
$Result

In [ ]:
# Get only the accesstoken
$Result.AccessToken

In [ ]:
# Retrive the permissions with the MSAL Library
$Result.Scopes

# Optinally: Inspect the accesstoken with JWTDetails
($Result.AccessToken | Get-JWTDetails).scp

In [ ]:
# Use the token in a Microsoft Graph Query v1.0

# Normal way
$Header = @{
    Authorization = "Bearer $($Result.AccessToken)"
}

# Msal.PS way
$Header_MSAL = @{
    Authorization = $Result.CreateAuthorizationHeader()
}

Write-Output "With classic header."
$users_normal = Invoke-RestMethod -Headers $Header -Uri  'https://graph.microsoft.com/v1.0/users' -Method Get
$users_normal.value | Select-Object -First 1

Write-Output "With MSAL created header"
$users_msal = Invoke-RestMethod -Headers $Header_MSAL -Uri  'https://graph.microsoft.com/v1.0/users' -Method Get
$users_msal.value | Select-Object -First 1

In [ ]:
# Use the token in a Microsoft Graph Query beta. See the difference?

$Header = @{
    Authorization = "Bearer $($Result.AccessToken)"
}

$users = Invoke-RestMethod -Headers $Header -Uri  'https://graph.microsoft.com/beta/users' -Method Get
$users.value | Select-Object -First 1

But how do I knew how to query users? That's where the Microsoft Graph explorer comes in. 
Hit the following url to get to It:

[Graph Explorer (https://developer.microsoft.com/en-us/graph/graph-explorer)](https://developer.microsoft.com/en-us/graph/graph-explorer)

If we do not find the query that we need here you can find It in the Microsoft Graph API Reference for sure [Graph API reference](https://docs.microsoft.com/en-us/graph/api/overview?view=graph-rest-beta)

For reference how to use query parameters inside of Microsoft Graph [Query Parameters](https://docs.microsoft.com/de-de/graph/query-parameters)

### Query Graph API from inside VSCode
---

The following prerequisites are needed:
- MSAL.PS PowerShell Module
- VSCode extension Rest Client
- Git Hub Repository from Ben Reader [VSCode Rest Client Demo](https://github.com/tabs-not-spaces/VSCode_RestClient_Demo)

### Use the Microsoft.Graph PowerShell Module